# Dependencies description

An example of how this looks in practice

In [1]:
import os

from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.dependencies import (InterDependencies, interdeps_to_yaml,
                                         yaml_to_interdeps)
from qcodes.dataset.measurements import Measurement

In [3]:
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])

In [4]:
meas = Measurement()
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1, dac.ch2))

### Burning question 1: how does it look, then, the yaml?

Let's first make a `InterDependencies` object for the `Measurement`

In [5]:
idp = InterDependencies(*tuple(meas.parameters.values()))

Okay, that was easy. So is making the yaml.

In [6]:
yaml_str = interdeps_to_yaml(idp)
print(yaml_str)

Parameters:
- !ParamSpec
  name: dac_ch1
  type: numeric
  label: Gate ch1
  unit: V
  _inferred_from: []
  _depends_on: []
- !ParamSpec
  name: dac_ch2
  type: numeric
  label: Gate ch2
  unit: V
  _inferred_from: []
  _depends_on: []
- !ParamSpec
  name: dmm_v1
  type: numeric
  label: Gate v1
  unit: V
  _inferred_from: []
  _depends_on:
  - dac_ch1
  - dac_ch2



### Burning question 2: okay, but do we all information back?

Let's read it back and see.

In [7]:
new_idp = yaml_to_interdeps(yaml_str)

In [8]:
new_idp.paramspecs

(dac_ch1 (numeric), dac_ch2 (numeric), dmm_v1 (numeric))

In [9]:
dmm_ps = new_idp.paramspecs[2]
dmm_ps

dmm_v1 (numeric)

We can compare this ParamSpec to the one from `meas`

By the way, we should really update `__repr__` of `ParamSpec` and implement/override `__eq__`.

In [10]:
dmm_v1 = meas.parameters['dmm_v1']

In [11]:
for attr in ['name', 'label', 'unit', '_depends_on', '_inferred_from']:

    print(getattr(dmm_ps, attr) == getattr(dmm_v1, attr))

True
True
True
True
True
